# Vectorization
Let's start of with a where we left of last time. Simple classification using a bag-of-words. Today we will only be using the multinomial naive bayes to keep the focus on the vectorization.

** Task go through the script in studygroups read through the script, make sure you understanding the working as you go through it.**

if you find any if this uninterpretable and or difficult, remember that I will be in the class or on zoom if you need help.

In [33]:
# add classification script to path (as well as data)
import sys
import os
path = os.path.join("..", "class_05")  # create path - will be different depending on mac vs windows

sys.path.append(path)  # add path

# create path for imdb dataset
imdbpath = os.path.join("..", "class_05", "imdb")
print(imdbpath)

..\class_05\imdb


In [34]:
from classification import read_imdb

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

# read data
imdb = read_imdb(imdbpath)

# train test split
X_train, X_test, y_train, y_test = train_test_split(imdb.text, imdb.tag)
text_clf = Pipeline([('vect', CountVectorizer()), 
                     ('clf', MultinomialNB())])

# train model
text_clf.fit(X_train, y_train)

# estimate performance
predictions = text_clf.predict(X_test)

acc = sum(predictions == y_test)/len(y_test)
print(f"Our model obtained a performance accuracy of {acc}")


Our model obtained a performance accuracy of 0.814


---
Now let is try to add our own tokenizer:

In [35]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mikkel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mikkel\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [36]:
# a simple test set:
doc = ["NLP is very very fun", 
       "NLP teachers are fun",
       "a teacher is a fun person",
       "NLP is a very fun person"]

from text_preprocessor import Text

# define a wrapper function which only returns tokens and handles list
def tokenization_wrapper(txt):
    TextObject = Text(txt)
    TextObject.tokenize(method="nltk", lemmatize = False)
    tokens = TextObject.get_tokens()
    return tokens


vectorizer = CountVectorizer(tokenizer=tokenization_wrapper, lowercase=False)
bow = vectorizer.fit_transform(doc)
bow.todense()
# TASK: Change the above code use your lemmatization function instead of pure tokenization. What would you expect would change?

matrix([[1, 0, 0, 1, 1, 0, 0, 0, 2],
        [1, 0, 1, 1, 0, 0, 0, 1, 0],
        [0, 2, 0, 1, 1, 1, 1, 0, 0],
        [1, 1, 0, 1, 1, 1, 0, 0, 1]], dtype=int64)

---
You can make this matrix more visually appealing (and understandable) quite well using this trick:

In [37]:
import pandas as pd
pd.DataFrame(bow.todense(), columns=vectorizer.get_feature_names())
# Notice that teacher and teachers are two different tokens is this problematic?
# TASK: the visualization code is a bit unclear to understand. Make a function on the form:
# vizualize(bow, vectorizer)
# which produces the table below

,NLP,a,are,fun,is,person,teacher,teachers,very
0,1,0,0,1,1,0,0,0,2
1,1,0,1,1,0,0,0,1,0
2,0,2,0,1,1,1,1,0,0
3,1,1,0,1,1,1,0,0,1


---
We now want to look a bit more into what goes on in the vectorization

In [38]:
# using 1-grams and bigrams:
vectorizer = CountVectorizer(ngram_range=(1, 2)) #(2,3) instead

bow = vectorizer.fit_transform(doc)
bow.todense()
# TASK change it to only use 2-grams and 3-grams

matrix([[0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 2, 1, 1],
        [1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0],
        [0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0],
        [0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0]],
       dtype=int64)

In [39]:
# to use stopwords
from nltk.corpus import stopwords
stopwordlist = stopwords.words('english')

vectorizer = CountVectorizer(stop_words=stopwordlist)
bow = vectorizer.fit_transform(doc)

pd.DataFrame(bow.todense(), columns=vectorizer.get_feature_names())

# TASK what words were removed using the stopwordlist, might these be meaningful?

# "very" could be used as a strength marker

,fun,nlp,person,teacher,teachers
0,1,1,0,0,0
1,1,1,0,0,1
2,1,0,1,1,0
3,1,1,1,0,0


In [40]:
# another way to remove stopwords
vectorizer = CountVectorizer(max_df=0.9) # remove words which appear in 90% of the documents
bow = vectorizer.fit_transform(doc)
bow.todense()

matrix([[0, 1, 1, 0, 0, 0, 2],
        [1, 0, 1, 0, 0, 1, 0],
        [0, 1, 0, 1, 1, 0, 0],
        [0, 1, 1, 1, 0, 0, 1]], dtype=int64)

In [41]:
# To make a binary classification (is the word there or not)
vectorizer = CountVectorizer(binary=True)
bow = vectorizer.fit_transform(doc)
bow.todense()


matrix([[0, 1, 1, 1, 0, 0, 0, 1],
        [1, 1, 0, 1, 0, 0, 1, 0],
        [0, 1, 1, 0, 1, 1, 0, 0],
        [0, 1, 1, 1, 1, 0, 0, 1]], dtype=int64)

In [42]:
# remove words which appear less than min_df (i.e. words which appear only once)
vectorizer = CountVectorizer(min_df=2)
bow = vectorizer.fit_transform(doc)
bow.todense()

# min_df can also be a number from 0-1. If so it is a percentage

matrix([[1, 1, 1, 0, 2],
        [1, 0, 1, 0, 0],
        [1, 1, 0, 1, 0],
        [1, 1, 1, 1, 1]], dtype=int64)

---
# Tf-Idf

tf-idf for a specific term (word) $t$ in a given document $d$, as you probably know from the lectures, is calculated using the

$tf-idf(t, d) = tf(t, d) * idf(t)$

Where $tf$ is the frequency of term $t$ in the document $d$.and $idf$ is the inverse document frequency of the term $t$. In sklearn $idf$ is calculated as follows:

$idf(t) = log \frac{1 + n}{1 + df(t)} + 1$

it is then normalized using the l2 norm (the euclidian norm for all math people out there). It simply makes sure it the length of vector for the document is one. It is calculated as follows:

$\frac{1}{\sqrt{v_1^2 + v^2 + ...}}$

As an example the norm of v would be:

$
  \begin{align}
    ||v|| &= ||\begin{bmatrix}
           1 \\
           1 \\
           0 \\
         \end{bmatrix}|| = \frac{1}{\sqrt{1 + 1 + 0}} = \frac{1}{\sqrt{2}} 
  \end{align}
  $


The tf idf in sklearn uses the default parameters:
```python
norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=False
```

where `norm` toggles what type of normalization it should use. `use_idf` toggles whether is should multiple tf with idf not not.
`smooth_idf` the is the `+1` fraction. i.e. setting it to false would do the following: 

$idf(t) = log \frac{n}{df(t)} + 1$

This is the same as the add one smoothing the naive bayes assignment.

Interestingly tf-idf can also be derived using information theory as **the amount of information gained seeing the word weighted by your probability of seeing it**. It is thus reasonable that it is good for performance, not only that, but it also fits nicely into cognitive theories such as bayesian brain and the free energy principles.

Read a lot more on tf-idf in sklearn [here](https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction).

There is two ways to call do tf-idf in scikit-learn. Either using the `TfidfVectorizer` or transforming the `CountVectorizer` using `TfidfTransformer`. I will just be using the first one for simplicity. You can pass `TfidfVectorizer` all the same arguments as we have used for the `CountVectorizer`.


In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
ti = tfidf.fit_transform(doc)
ti.todense()
pd.DataFrame(ti.todense(), columns=tfidf.get_feature_names())

,are,fun,is,nlp,person,teacher,teachers,very
0,0.000000,0.276052,0.337651,0.337651,0.000000,0.000000,0.000000,0.834133
1,0.610878,0.318782,0.000000,0.389916,0.000000,0.000000,0.610878,0.000000
2,0.000000,0.343993,0.420753,0.000000,0.519714,0.659191,0.000000,0.000000
3,0.000000,0.341846,0.418127,0.418127,0.516470,0.000000,0.000000,0.516470


In [44]:
# we can add all the previous stuff as well
tfidf = TfidfVectorizer(tokenizer=tokenization_wrapper,
                        stop_words=stopwordlist,
                        min_df=1,
                        max_df=0.8,
                        ngram_range=(1,2),
                        lowercase=False,
                        binary=False
                        )
ti = tfidf.fit_transform(doc)
ti.todense()
pd.DataFrame(ti.todense(), columns=tfidf.get_feature_names())

,NLP,NLP fun,NLP teachers,fun person,person,teacher,teacher fun,teachers,teachers fun
0,0.629228,0.777221,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.345783,0.000000,0.541736,0.000000,0.000000,0.000000,0.000000,0.541736,0.541736
2,0.000000,0.000000,0.000000,0.437791,0.437791,0.555283,0.555283,0.000000,0.000000
3,0.423442,0.523035,0.000000,0.523035,0.523035,0.000000,0.000000,0.000000,0.000000


# Dimensionality reduction
As the matrix for language classification typically becomes quite large an idea is to reduce the dimensionality of the matrix. Typically used techniques reducing dimensionality in data include Principal component analysis (PCA) or singular value decompositions (SVD). Note that PCA was introduced 2nd semester Experimental Methods so people should be familiar (alternatively there is also a video in the highly recommended readings for class). PCA is also used in computing the intelligence metric, g-factor. PCA is calculated using SVD which, for the mathematically inclined, is the generalization of eigendecompositions (finding eigenvalues and eigenvectors) to non-diagnizable matrices. 

Other approaches include latent semantic analysis (LSA) which similar to PCA is build upon SVD. LSA was later reinterpreted in a probalistic framework (pLSA) which was generalized into Latent Dirichlet Allocation (LDA) which is what is the topic model we will go into in week 44. LSA is also more efficient for tf-idf and tf matrices.



In [46]:
from sklearn.decomposition import PCA

tfidf = TfidfVectorizer(ngram_range=(1,3))
ti = tfidf.fit_transform(doc)
pca = PCA(n_components=3) # should be bigger for real data
output = pca.fit_transform(ti.todense())  # it is required to make the tf-idf matrix (ti) to dense to do PCA, but this is computionally heavy (thus the LSA)

print("the explained variance in the dataset of each of the three components:")
print(pca.explained_variance_.round(3)) 

the explained variance in the dataset of each of the three components:
[0.361 0.317 0.13  0.   ]


In [47]:
from sklearn.decomposition import TruncatedSVD as LSA
lsa = LSA(n_components=4) # should be bigger for real data
output = lsa.fit_transform(ti)  # notice the lack of todense

print("the explained variance in the dataset of each of the three components:")
print(lsa.explained_variance_.round(2)) 
print(sum(lsa.explained_variance_)) 

print("\nthe reduced matrix:")
print(output)

print("\nthe previous matrix:")
print(ti.todense())
# notice how many of the entries in the tf-idf (similar to the bow matrices) contain a lot of zeros, which indicate that the information can typically be reduced dramatically.


the explained variance in the dataset of each of the three components:
[0.07 0.22 0.22 0.1 ]
0.60633589712259

the reduced matrix:
[[ 0.81804119 -0.0492465  -0.40425051 -0.40615873]
 [ 0.23147691  0.96072669  0.15301126 -0.0031986 ]
 [ 0.44846932 -0.20639604  0.85867458 -0.13767322]
 [ 0.88287133 -0.10141692 -0.10172966  0.44710613]]

the previous matrix:
[[0.         0.         0.17337136 0.         0.21205806 0.
  0.         0.2619339  0.         0.33222994 0.21205806 0.2619339
  0.2619339  0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.5238678  0.2619339
  0.         0.33222994 0.33222994]
 [0.36085029 0.36085029 0.18830665 0.         0.         0.
  0.         0.         0.         0.         0.23032606 0.
  0.         0.36085029 0.36085029 0.         0.         0.
  0.         0.36085029 0.36085029 0.36085029 0.         0.
  0.         0.         0.        ]
 [0.         0.         0.19833261 0.29964599 0.24258925 0.38006294
  0.3800

---
# Tasks
1) putting it all together. Create a pipeline which does the both the vectorization, dimensionality reduction and the classification using the imdb dataset. Set the original test set aside as a validation set and make a new train test split.
here the train is for training on. Test is for testing performance and the validation set is to validate performance (which we will do in 7)

2) Change the preprocessing to get a better performance (don't spent to long here, just a little bit of tuning)

3) Apply the same model to the 20 newsgroup data (see next block on how to read it in). Again split up the train set into a train and test set. leave the actual test set i.e. validation set for 7 (loaded in by setting `subset="test"`).

4) apply the same model to the sms spam data from class 4. (spam.csv). Remember to split it into a validation set and a train set and then again split the train set into a train and a test set.

5) When you finetune your model now do you see that you preprocessing choices you made were only good for specific types of dataset  or did they generalize well? Can you find general trends with works better than others? E.g. Is stopwords a good idea? Does tf-idf outperform bag of words? Found out 5 of such 'truths' and post then on element

6) Finally finetune yours models. Optimize the vectorization as much as you want to. Use cross validation or grid search on the train/test set. Until you are satisfied. (it does not have to be the same preprocessings steps for all three dataset anymore)

7) Finally apply your model to validation sets and report your models performance on element. It should have the form:

*"We obtained a performance of XX% on validation set of (the dataset) using (your model specifications)..."*

In [49]:
# your code here (or in another document)

def read_imdb(imdb_path="imdb"):
    """
    TASK 1: Write a short description about what this function does and how it
    does it. You should also describe the structure of imdb_files.
    """
    imdb_files = []
    for path, subdirs, files in os.walk(imdb_path):
        for f in files:
            filepath = os.path.join(path, f)

            with open(filepath) as f:
                text = f.read()

            tag = os.path.split(path)[1]
            imdb_files.append([tag, text])
    return pd.DataFrame(imdb_files, columns=["tag", "text"])

imdb = read_imdb()

,tag,text
0,neg,"plot : two teen couples go to a church party ,..."
1,neg,the happy bastard's quick movie review \ndamn ...
2,neg,it is movies like these that make a jaded movi...
3,neg,""" quest for camelot "" is warner bros . ' firs..."
4,neg,synopsis : a mentally unstable man undergoing ...


In [72]:
from sklearn.preprocessing import MinMaxScaler

X_train, X_val, y_train, y_val = train_test_split(imdb.text, imdb.tag, test_size = 0.1)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train)

text_clf = Pipeline([('vect', TfidfVectorizer(ngram_range=(1,3))),
                    ('dr', LSA(n_components=100)),
                    ('minmax', MinMaxScaler()),
                     ('clf', MultinomialNB())])

# train model
text_clf.fit(X_train, y_train)


# estimate performance
predictions = text_clf.predict(X_test)

acc = sum(predictions == y_test)/len(y_test)
print(f"Our model obtained a performance accuracy of {acc}")

Our model obtained a performance accuracy of 0.6066666666666667


---
## reading in the 20 newsgroup data

In [50]:
from sklearn.datasets import fetch_20newsgroups

# select which categories to load
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']
twenty_train = fetch_20newsgroups(subset='train', # only load the training set
                                  categories=categories)

# transform the data (not necessary but nice)
df = pd.DataFrame({"text": twenty_train.data, "targets": twenty_train.target})
df["category"] = df.targets.apply(lambda x: categories[x])

# examine the data (first five rows)
df.head()

# SUGGESTION: loading dataset like this can be hard to read later on. Wrapping it all into a function e.g. read_20_news make the code easy to read an easy to reuse later.

,text,targets,category
0,From: sd345@city.ac.uk (Michael Collier)\nSubj...,1,soc.religion.christian
1,From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\...,1,soc.religion.christian
2,From: djohnson@cs.ucsd.edu (Darin Johnson)\nSu...,3,sci.med
3,From: s0612596@let.rug.nl (M.M. Zwart)\nSubjec...,3,sci.med
4,From: stanly@grok11.columbiasc.ncr.com (stanly...,3,sci.med
